In [ ]:
# list of subreddits to collect posts from
sub_list = ['greentext', 'deepfriedmemes', 'me_irl', '2meirl4meirl', 'dankmemes', 
            'adviceanimals', 'dogelore', 'wholesomememes', 'CatsPlayingDnd', 'glitch_art', 
            'awwnime', 'EarthPorn', 'tumblr', 'NapkinMemes', 'chickenswearingpants', 
            'BreadStapledToTrees', 'terriblefacebookmemes', 'Animemes', 'dataisbeautiful']

In [ ]:
# PRAW methodology
# import praw

In [ ]:
# PRAW methodology
# reddit = praw.Reddit(
#     site_name='scraperbot')

In [ ]:
# PRAW methodology
# def scrape_sub(count_each, scraper_funcs=[]):
#     data = {}
#     for func in scraper_funcs:
#         for post in func(limit=count_each):
#             if not post.over_18:
#                 if post.url[-4:] == '.jpg':
#                     data[post.id] = (post.title, post.url, post.subreddit.display_name, post.created_utc, post.score, post.upvote_ratio)
#     return data

In [ ]:
# PRAW methodology
# data = {}

# for sub_name in sub_list:
#     subreddit = reddit.subreddit(sub_name)
#     data.update(scrape_sub(1000, scraper_funcs=[subreddit.hot, subreddit.new, subreddit.rising, subreddit.top]))

In [ ]:
import urllib.request
import json
import pandas as pd
import numpy as np
import os
import pickle
pickle.HIGHEST_PROTOCOL = 4

In [ ]:
def filter_posts(post_df):
    output_df = post_df
    output_df = output_df[output_df.is_self == False]
    output_df = output_df[output_df.over_18 == False]
    output_df = output_df[output_df.pinned == False]
    output_df = output_df[output_df.score >= 5]
    output_df = output_df[output_df.url.str.endswith('.jpg', na=False)]
    return output_df

In [ ]:
# Collect approximately `min_posts` number of posts (or the total # of posts in subreddit, whichever is smaller)
# Collect posts from `subreddit`
# Output the data file to `output_filename`
def scrape_subreddit(min_posts, subreddit):
    # get the sub creation date
    with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=1&sort=asc&before=30d") as url:
        data = json.loads(url.read().decode())
    df_sub = pd.DataFrame.from_dict(pd.json_normalize(data['data']), orient='columns')
    created_utc_first_sub = df_sub.tail(1)['created_utc']

    # get the most recent N posts
    with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=1&sort=desc&before=30d") as url:
        data = json.loads(url.read().decode())
    df_sub = pd.DataFrame.from_dict(pd.json_normalize(data['data']), orient='columns')
    created_utc_now_sub = df_sub.tail(1)['created_utc']
    
    # make sure we get at least args.post_count number of posts scraped
    # but also don't go past the beginning of the subreddit
    try:
        while (len(df_sub.index) < min_posts) and (int(created_utc_now_sub) > int(created_utc_first_sub)): 
            with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=1000&sort=desc&before=%d"%created_utc_now_sub) as url:
                data = json.loads(url.read().decode())
            df_new_sub = pd.DataFrame.from_dict(pd.json_normalize(data['data']), orient='columns')
            df_sub = df_sub.append(df_new_sub)
            created_utc_now_sub = df_sub.tail(1)['created_utc']
            df_sub = filter_posts(df_sub)
            print(len(df_sub))
    finally:
        # Clean up the post data that we scraped
        df_sub = filter_posts(df_sub)
        df_sub = df_sub.set_index('id')
        df_sub = df_sub[~df_sub.index.duplicated(keep='first')]
        df_sub = df_sub[['author', 'author_flair_text', 'created_utc', 'permalink', 'retrieved_on', 
                         'score', 'subreddit', 'title', 'upvote_ratio', 'url']]
        return df_sub

In [ ]:
# subreddit = 'deepfriedmemes'
# with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=100&sort=desc&before=30d") as url:
#     data = json.loads(url.read().decode())
# df_sub = pd.DataFrame.from_dict(pd.json_normalize(data['data']), orient='columns')
# df_sub[['over_18', 'score', 'title', 'url', 'permalink']]

In [ ]:
dataset_df = pd.DataFrame()

for sub_name in ['me_irl']:#sub_list:
    scraped_df = scrape_subreddit(101, sub_name)
    print('final grab', len(scraped_df))
    dataset_df = pd.concat([dataset_df, scraped_df])
    dataset_df = dataset_df.append(scraped_df)
# dataset_df.to_hdf('data/dataset.h5', key='df')

In [ ]:
pd.read_hdf('data/dataset.h5', 'df')

In [ ]:
df = pd.read_hdf('data/dataset.h5', 'df')
df

In [ ]:
len(df[df.subreddit.str.lower() == 'DeepFriedMemes'.lower()])

In [ ]:
df = df[~df.index.duplicated(keep='first')]
df = df[['author', 'author_flair_text', 'created_utc', 'permalink', 'retrieved_on',
         'score', 'subreddit', 'title', 'upvote_ratio', 'url']]
df

In [ ]:
if os.path.isfile('data/dataset.h5'):
    os.remove('data/dataset.h5')
df.to_hdf('data/dataset.h5', key='df')